# COGS 108 - Data Checkpoint

# Names

- Joonsung Park
- James Chung
- Richard Gross
- Madison Hambly
- Colin Lintereur

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [32]:
import numpy as np
import requests
# import io
# from pathlib import Path
import json
import zipfile
import pandas as pd
import re

import paralleldots
paralleldots.set_api_key("aSWrD1QSsUqdb8JYWJ6jMJMXrTxjFXKhdD4xQZKiUlQ")

# # For single sentence
# text="Apple was founded by Steve Jobs."
# response=paralleldots.taxonomy(text)
# print(response)

# # for multiple sentence as array
# text=["Apple was founded by Steve Jobs.","Apple Inc. is an American multinational technology company headquartered in Cupertino, California"]
# response=paralleldots.batch_taxonomy(text)
# print(response)

# regex = r"[$0-9^]{2,}"
# test_str = "FinalProject_group050_S.ipynb"
# matches = re.search(regex, test_str)
# print(matches[0])

research_questions = {}
qtr = []
year = []
group = []
topic = []
special = []
word_counts = []
index = 0

with zipfile.ZipFile("zips/FinalProjects-Wi21-main.zip", "r") as f:
    for i, name in enumerate(f.namelist()):
        if '.ipynb' not in name:
            continue
        rf = json.loads(f.read(name))
        word_count = 0
        rq_found = False
        for j, cell in enumerate(rf['cells']):
            if cell['cell_type'] == 'markdown':
                for c in cell['source']:
                    if (rq_found == False) and (re.search(r"(^#.*research question)", c.lower()) != None):
                        qtr.append("WI")
                        year.append(2021)
                        group.append(int(re.search(r"[$0-9^]{3,}", name)[0]))
                        special.append(False if re.search(r"_S\.ipynb", name) == None else True)
                        rq = ' '.join(rf['cells'][j+1]['source'])
                        research_questions[str(group[index])+"_"+qtr[index]+"_"+str(year[index])] = rq          
                        rq_found = True
                        index += 1
                    word_count += len(c.replace('#', '').lstrip().split(' '))
        word_counts.append(word_count)

In [33]:
topic = []
for i, (g, q, y) in enumerate(zip(group, qtr, year)):
    #topic.append(input(research_questions[group[i]+"_"+qtr[i]+"_"+year[i]]))
    response = paralleldots.taxonomy(research_questions[str(group[i])+"_"+qtr[i]+"_"+str(year[i])])
    topic.append(response['taxonomy'][0]['tag'])
    print(research_questions[str(group[i])+"_"+qtr[i]+"_"+str(year[i])], " :: ", topic[i])

*What is the relative effect of unemployment, poverty, and homelessness on adult obesity rates (BMI >= 30) across states in the United States?*  ::  IMPACT
Was there a relationship between the median age and the median cost of living across congressional districts in a given state in 2019?  ::  POLITICS
How did the 2019-20 NBA Bubble affect players' shooting performance in the 2019-20 playoffs compared to that of the previous 5 years (2014-15 to 2018-19)?  ::  SPORTS
Is there a relationship between the search popularity (in terms of Google searches) a movie receives before its release and the box-office revenue it makes during its opening weekend?  ::  ENTERTAINMENT
What are the factors that contribute to the success of video game sales across the regions of North America, Europe, and Japan?
  ::  POLITICS
**Is there a relationship between the frequency of mentions of Tesla, Apple, and Zoom on Reddit and Twitter and the change in their respective average weekly closing prices from 01/0

How has the adoption of ridesharing affected the GHG emissions in a populous city with a pre-existing reliance on car services like New York City?   ::  POLITICS
On average, do international-born NBA players historically perform better than domestic player?  ::  SPORTS
How does treatment method (invasive or noninvasive) in bone fracture injuries correlate to game performance before and after treatment in NBA players?  ::  HEALTHY LIVING
During 2020 and the pandemic was there a significant increase in 'sad' songs compared to years prior?  ::  IMPACT
What is the trajectory of future air pollution and temperatures in California counties based on past trends of California climate?  ::  HEALTHY LIVING
Can we predict which state in the USA will be covid-free first based on current hospital records, vaccination rates, and population?  ::  HEALTHY LIVING
Are there significant differences among courses under the Humanities, Social Science, and STEM departments at UCSD in terms of correlation be

*Did the amount of politically charged Tweets (i.e. Tweets containing political keywords like election, voting, president) increase in 2020 relative to 2016 and did this reflect a change in new voter registration between these two election years?*  ::  POLITICS
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;To what extent did tweets that directly reference Donald Trump and/or Joe Biden contribute to voter turnout by state in the 2020 US presidential election?  ::  POLITICS
How has COVID-19 preventative measures affected the amounts of and rates of PPE pollution collected on the coastlines of the United States over time?  ::  HEALTHY LIVING


In [34]:
print(len(group), len(qtr), len(year), len(topic), len(special), len(word_counts))
d = {'Group':group, 'Quarter':qtr, 'Year':year, 'Topic':topic, 'Special':special, 'Word_Count':word_counts}
#df = pd.DataFrame(list(zip(group, qtr, year, topic, special, word_counts)), columns =['Group', 'Qtr', 'Year', 'Topic', 'Special', 'Word_Count'])
df = pd.DataFrame(data = d)
output = df.to_csv(path_or_buf="./"+qtr[0] + str(year[0])+".csv")
print(output)

65 65 65 65 65 65
None


# Data Cleaning

Describe your data cleaning steps here.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION